In [1]:
import tensorflow as tf

/home/yoichi/.pyenv/versions/anaconda2-4.3.0/envs/choco3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Importing Data
- tf.Data APIはシンプルなピースで複雑な入力パイプラインを構築できる
    - 例えば、分散ファイルシステムを使っている場合に、複数のファイルを集めてランダムなノイズを乗せたり、ミニバッチ用にランダムにセレクトするなどが必要な場合ががある。
    - 他の例では、テキストデータの場合、生のテキストデータからシンボルを抽出したり云々。
- tf.data API は二つの新しい概念を提供する
    - tf.data.Datasetは一連の要素を表す。ここでの要素は、1つ以上のTensorオブジェクトが含まれる。例えば、image pipelineの場合、画像データとラベルを表すテンソルのペア。以下の二つの方法がある
        - source(e.g. Dataset.from_tensor_slices()):一つ以上のtf.Tensorオブジェクトからsetasetを生成する
        - transformation(e.g. Dataset.batch()):一つ以上のtf.data.Datasetオブジェクトから
    - tf.data.Iteratorはdatasetから要素を抽出する主要な方法を示す。Iterator.get_next()によって、Datasetの次の要素が返される。

https://www.tensorflow.org/guide/datasets

https://www.tensorflow.org/versions/r1.8/programmers_guide/datasets

## Basic mechanism
様々な種類のDatasetオブジェクトとIteratorオブジェクトを作成する基本と、それらからデータを抽出する方法についての解説.

- まずデータソースを定義する必要がある
    - 例えば、メモリ上のいくつかのテンソルからDatasetを生成するには、tf.data.Dataset.from_tensors() or tf.data.Dataset.from_tensor_slices()を使う。
    - または、TFRecord formatでdiskにあるなら、tf.data.TFRecordDataset。

- Datasetオブジェクトを作成したら、tf.data.Datasetオブジェクトのメソッドを連させて新しいDatasetに変換できる
    - 例えば、Dataset.map(各要素に関数を適用する), Dataset.batch(複数要素変換?)

- Datasetからデータを消費する方法は、iteratorを作成すること(tf.data.Iterator)
    - Iterator.initializer:iteratorの状態を初期化する
    - Iterator.get_next():tf.Tensorオブジェクトを返す。

### Dataset structure
- datasetは同じ構造の要素の集合で構成される。これらの要素は一つ以上のtf.Tensorオブジェクトでコンポーネントと呼ぶ。
- 各コンポーネントはtf.Dtype(テンソルの型を表す)とtf.TensorShape(各要素の形状)を持つ。
- Dataset.output_types and Dataset.output_shapes プロパティはデータセット要素の各コンポーネントの推定される型と形状を調べるのに使う


In [2]:
# 単一コンポーネント([size, dimensions])
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10]))
print(dataset1.output_types)  # ==> "tf.float32"
print(dataset1.output_shapes)  # ==> "(10,)"

<dtype: 'float32'>
(10,)


In [3]:
# 複数のコンポーネントで構成する
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random_uniform([4]),
     tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)))
print(dataset2.output_types)  # ==> "(tf.float32, tf.int32)"
print(dataset2.output_shapes)  # ==> "((), (100,))"

(tf.float32, tf.int32)
(TensorShape([]), TensorShape([Dimension(100)]))


In [4]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
print(dataset3.output_types)  # ==> (tf.float32, (tf.float32, tf.int32))
print(dataset3.output_shapes)  # ==> "(10, ((), (100,)))"

(tf.float32, (tf.float32, tf.int32))
(TensorShape([Dimension(10)]), (TensorShape([]), TensorShape([Dimension(100)])))


- 各コンポーネントに名前をつけることができる
- 辞書の形で名前を指定できる

In [5]:
dataset = tf.data.Dataset.from_tensor_slices(
    {"a": tf.random_uniform([4]),
     "b": tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)})
print(dataset.output_types)  # ==> "{'a': tf.float32, 'b': tf.int32}"
print(dataset.output_shapes)  # ==> "{'a': (), 'b': (100,)}"

{'a': tf.float32, 'b': tf.int32}
{'a': TensorShape([]), 'b': TensorShape([Dimension(100)])}


- datasetの変形(変換)
- Dataset.map, Dataset.flat_map, Dataset.filter

```
dataset1 = dataset1.map(lambda x: ...)

dataset2 = dataset2.flat_map(lambda x, y: ...)

dataset3 = dataset3.filter(lambda x, (y, z): ...)
```

### Creating an iterator
- Datasetを作ったら、次は、各要素にアクセスするためのIteratorを作成する
- 4つのiteratorをサポートしている


#### one-shot
- 最もシンプルなiterator
- datasetを一回だけ反復する
- Estimatorで簡単に利用できる唯一の型

In [9]:
dataset = tf.data.Dataset.range(100)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

sess = tf.Session()
for i in range(100):
    value = sess.run(next_element)
    assert i == value

#### initializable
- 使用する前にiterator.initializerを呼び出す必要がある
- データセットの定義をiteratorを初期化するときに供給される一つ以上のtf.placeholderテンソルを利用してパラメとライズできる。

In [12]:
max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Initialize an iterator over a dataset with 10 elements.
sess.run(iterator.initializer, feed_dict={max_value: 10})
for i in range(10):
    value = sess.run(next_element)
    print(value)
    assert i == value

0
1
2
3
4
5
6
7
8
9


In [15]:
# Initialize the same iterator over a dataset with 100 elements.
sess.run(iterator.initializer, feed_dict={max_value: 100})
for i in range(100):
    value = sess.run(next_element)
    assert i == value
print(value)

99


#### reinitializable
- 複数の異なるDatasetオブジェクトから初期化できる
- 例えば、訓練データにランダムな摂動を載せたデータセットと評価用に修正しない画像のデータセットを入力する場合がある。

In [16]:
# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(100).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64))
validation_dataset = tf.data.Dataset.range(50)

In [17]:
training_dataset.output_types

tf.int64

In [18]:
training_dataset.output_shapes

TensorShape([])

In [19]:
# A reinitializable iterator is defined by its structure. We could use the
# `output_types` and `output_shapes` properties of either `training_dataset`
# or `validation_dataset` here, because they are compatible.
iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                           training_dataset.output_shapes)
next_element = iterator.get_next()

training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

# Run 20 epochs in which the training dataset is traversed, followed by the
# validation dataset.
for _ in range(20):
    # Initialize an iterator over the training dataset.
    sess.run(training_init_op)
    for _ in range(100):
        sess.run(next_element)

    # Initialize an iterator over the validation dataset.
    sess.run(validation_init_op)
    for _ in range(50):
        sess.run(next_element)

#### feedable
- feedable iteratorをtf.placeholderと共に使用するとtf.Session.runの各呼び出しで使用するイテレータを選択できます。
- reinitializable iteratorと同じ機能を提供するが、イテレーターを切り替えるときに初期化する必要がない

In [20]:
# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(100).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64)).repeat()
validation_dataset = tf.data.Dataset.range(50)

In [21]:
# A feedable iterator is defined by a handle placeholder and its structure. We
# could use the `output_types` and `output_shapes` properties of either
# `training_dataset` or `validation_dataset` here, because they have
# identical structure.
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

In [22]:
# You can use feedable iterators with a variety of different kinds of iterator
# (such as one-shot and initializable iterators).
training_iterator = training_dataset.make_one_shot_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()


In [23]:
# The `Iterator.string_handle()` method returns a tensor that can be evaluated
# and used to feed the `handle` placeholder.
training_handle = sess.run(training_iterator.string_handle())
validation_handle = sess.run(validation_iterator.string_handle())


In [25]:
# Loop forever, alternating between training and validation.
while True:
    # Run 200 steps using the training dataset. Note that the training dataset is
    # infinite, and we resume from where we left off in the previous `while` loop
    # iteration.
    for _ in range(200):
        sess.run(next_element, feed_dict={handle: training_handle})

    # Run one pass over the validation dataset.
    sess.run(validation_iterator.initializer)
    for _ in range(50):
        sess.run(next_element, feed_dict={handle: validation_handle})
    
    break

### Consuming values from an iterator
- Iterator.getnext()メソッドはイテレーターの次の要素に対応するtf.Tensorオブジェクトを返す
- 最後の要素でget_next()メソッドを実行すると、tf.errors.OutOfRangeErrorが返される
- 再度このIteratorを使いたい場合には、initializeが必要

In [26]:
dataset = tf.data.Dataset.range(5)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Typically `result` will be the output of a model, or an optimizer's
# training operation.
result = tf.add(next_element, next_element)

sess.run(iterator.initializer)
print(sess.run(result))  # ==> "0"
print(sess.run(result))  # ==> "2"
print(sess.run(result))  # ==> "4"
print(sess.run(result))  # ==> "6"
print(sess.run(result))  # ==> "8"


0
2
4
6
8


In [27]:
try:
    sess.run(result)
except tf.errors.OutOfRangeError:
    print("End of dataset")  # ==> "End of dataset"

End of dataset


- datasetの各要素が入れ子構造であるなら、Iterator.get_next()の戻り値は同じ入れ子構造のtf.Tensorオブジェクト
- 以下の例では、next1,2,3はIterator.get_next()で生成される同じオペレーションで生成されるテンソル。そのため、これらのテンソルのどれかを評価すると全てのコンポーネントのイテレータが前進する

In [28]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10]))
dataset2 = tf.data.Dataset.from_tensor_slices((tf.random_uniform([4]), tf.random_uniform([4, 100])))
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

iterator = dataset3.make_initializable_iterator()

sess.run(iterator.initializer)
next1, (next2, next3) = iterator.get_next()

In [29]:
next1

<tf.Tensor 'IteratorGetNext_10:0' shape=(10,) dtype=float32>

## Reading input data
### Consuming NumPy arrays
- 全てのデータがメモリにあるなら、tf.Tensorに変換しDataset.from_tensor_slices()を使って簡単にDatasetを作ることができる
- 以下の方法は、tf.constantでデータを定義する。これは、少ないデータなら有効

```
# Load the training data into two NumPy arrays, for example using `np.load()`.
with np.load("/var/data/training_data.npy") as data:
  features = data["features"]
  labels = data["labels"]

# Assume that each row of `features` corresponds to the same row as `labels`.
assert features.shape[0] == labels.shape[0]

dataset = tf.data.Dataset.from_tensor_slices((features, labels))
```

- tf.placeholder()を使ってDatasetを定義し、numpy arrayを入力することもできる

```
# Load the training data into two NumPy arrays, for example using `np.load()`.
with np.load("/var/data/training_data.npy") as data:
  features = data["features"]
  labels = data["labels"]

# Assume that each row of `features` corresponds to the same row as `labels`.
assert features.shape[0] == labels.shape[0]

features_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
# [Other transformations on `dataset`...]
dataset = ...
iterator = dataset.make_initializable_iterator()

sess.run(iterator.initializer, feed_dict={features_placeholder: features,
                                          labels_placeholder: labels})
```


### Consuming TFRecord data
- tf.data APIはメモリに載り切らない大きいデータセットも扱うことができる
- TFRecordフォーマットは行指向のバイナリフォーマット
- tf.data.TFRecordDatasetクラスは一つ以上のTFRecordファイルを入力パイプラインの一部として扱える

```
# Creates a dataset that reads all of the examples from two files.
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
```

- placeholderでファイル名を入力するノードを作っても良い
    - training用とvalidation用で分けるなど

### Consuming text data
- tf.data.TextLineDataset は一つ以上のテキスト形式のデータファイルからデータレコード(行)を抽出できる、簡単な方法
- TextLineDatasetは、1つ以上のファイル名を指定すると、それらのファイルの1行につき1つの文字列値要素を生成する
- default動作では、全てのファイルの全ての行を読みだすが、ヘッダーやコメントなどをスキップしたい場合もある。Dataset.skip() and Dataset.filter()を使ってこれらの行を飛ばすことができる。

```
filenames = ["/var/data/file1.txt", "/var/data/file2.txt"]
dataset = tf.data.TextLineDataset(filenames)
```

## Preprocessing data with Dataset.map()
- Dataset.map(f)は関数fをdatasetの各要素に適用することで、あたらしいデータセットを生成する

### Parsing tf.Example protocol buffer messages

### Decoding image data and resizing it
- resizeする

### Applying arbitrary Python logic with tf.py_func()
- パフォーマンスの理由で、データの前処理はできるだけTensorFlowを使用することが望ましい。
- しかし、外部ライブラリを呼び出すことが便利なこともある
- このとき、Dataset.map()でtf.py_func()を呼び出す

## Batching dataset elements

### Simple batching
- バッチ処理の最も簡単形式は、連続するn個の要素をスタックする


In [32]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0, -100, -1)
dataset = tf.data.Dataset.zip((inc_dataset, dec_dataset))
batched_dataset = dataset.batch(4)

iterator = batched_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

print(sess.run(next_element))  # ==> ([0, 1, 2,   3],   [ 0, -1,  -2,  -3])
print(sess.run(next_element))  # ==> ([4, 5, 6,   7],   [-4, -5,  -6,  -7])
print(sess.run(next_element))  # ==> ([8, 9, 10, 11],   [-8, -9, -10, -11])

(array([0, 1, 2, 3]), array([ 0, -1, -2, -3]))
(array([4, 5, 6, 7]), array([-4, -5, -6, -7]))
(array([ 8,  9, 10, 11]), array([ -8,  -9, -10, -11]))


### Batching tensors with padding
- 上の方法は、全て同じサイズのテンソルの場合に使える
- 異なるサイズのデータを扱えるモデルがあるので(時系列モデル等)、異なるサイズのデータをバッチ内でパディングして長さを揃えたい場合がある


In [33]:
dataset = tf.data.Dataset.range(100)
dataset = dataset.map(lambda x: tf.fill([tf.cast(x, tf.int32)], x))
dataset = dataset.padded_batch(4, padded_shapes=[None]) # バッチ内でパディング

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

print(sess.run(next_element))  # ==> [[0, 0, 0], [1, 0, 0], [2, 2, 0], [3, 3, 3]]
print(sess.run(next_element))  # ==> [[4, 4, 4, 4, 0, 0, 0],
                               #      [5, 5, 5, 5, 5, 0, 0],
                               #      [6, 6, 6, 6, 6, 6, 0],
                               #      [7, 7, 7, 7, 7, 7, 7]]

[[0 0 0]
 [1 0 0]
 [2 2 0]
 [3 3 3]]
[[4 4 4 4 0 0 0]
 [5 5 5 5 5 0 0]
 [6 6 6 6 6 6 0]
 [7 7 7 7 7 7 7]]


## Training workflow

### Processing multiple epochs
- 複数エポック学習するために、データセットを反復処理する方法
- Dataset.repeat()を利用する
- 引数無しでDataset.repeat()を利用すると、無期限に繰り返される

```
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)
dataset = dataset.repeat(10)
dataset = dataset.batch(32)
```

In [40]:
inc_dataset = tf.data.Dataset.range(10)
inc_dataset = inc_dataset.repeat(3)
batched_dataset = inc_dataset.batch(5)

iterator = batched_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

while True:
    try:
        print(sess.run(next_element))
    except tf.errors.OutOfRangeError:
        print("End of dataset")  # ==> "End of dataset"
        break


[0 1 2 3 4]
[5 6 7 8 9]
[0 1 2 3 4]
[5 6 7 8 9]
[0 1 2 3 4]
[5 6 7 8 9]
End of dataset


### Randomly shuffling input data
- Dataset.shuffle()を適用すると、入力datasetがランダムにシャッフルされる
- tf.RandomShuffleQueueと同じアルゴリズム
    - 固定サイズのバッファを保持し、そのバッファからランダムに次の要素を一様に選択

In [44]:
inc_dataset = tf.data.Dataset.range(10)
inc_dataset = inc_dataset.shuffle(buffer_size=1000)
inc_dataset = inc_dataset.repeat(3)
batched_dataset = inc_dataset.batch(5)

iterator = batched_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

while True:
    try:
        print(sess.run(next_element))
    except tf.errors.OutOfRangeError:
        print("End of dataset")  # ==> "End of dataset"
        break


[1 0 8 6 3]
[5 4 9 7 2]
[4 8 5 0 3]
[2 1 7 6 9]
[4 0 3 1 9]
[7 6 5 8 2]
End of dataset
